In [378]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from category_encoders import OrdinalEncoder

In [379]:
path_d = '/Users/bumsoojoe/Desktop/Section4_project/csv/diabetes_prediction_dataset.csv'
path_m = '/Users/bumsoojoe/Desktop/Section4_project/csv/medical.csv'

In [380]:
dliabetes = pd.read_csv(path_d)
medical = pd.read_csv(path_m)

In [381]:
dliabetes.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


In [382]:
medical.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


# 당뇨 EDA
- taget : diabetes
-

In [383]:
dliabetes.shape

(100000, 9)

In [384]:
dliabetes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   gender               100000 non-null  object 
 1   age                  100000 non-null  float64
 2   hypertension         100000 non-null  int64  
 3   heart_disease        100000 non-null  int64  
 4   smoking_history      100000 non-null  object 
 5   bmi                  100000 non-null  float64
 6   HbA1c_level          100000 non-null  float64
 7   blood_glucose_level  100000 non-null  int64  
 8   diabetes             100000 non-null  int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 6.9+ MB


In [385]:
dliabetes.isnull().sum()

gender                 0
age                    0
hypertension           0
heart_disease          0
smoking_history        0
bmi                    0
HbA1c_level            0
blood_glucose_level    0
diabetes               0
dtype: int64

In [386]:
dliabetes.columns

Index(['gender', 'age', 'hypertension', 'heart_disease', 'smoking_history',
       'bmi', 'HbA1c_level', 'blood_glucose_level', 'diabetes'],
      dtype='object')

In [387]:
dliabetes.duplicated().sum()

3854

In [388]:
dliabetes[dliabetes.duplicated()].head(10)

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
2756,Male,80.0,0,0,No Info,27.32,6.6,159,0
3272,Female,80.0,0,0,No Info,27.32,3.5,80,0
3418,Female,19.0,0,0,No Info,27.32,6.5,100,0
3939,Female,78.0,1,0,former,27.32,3.5,130,0
3960,Male,47.0,0,0,No Info,27.32,6.0,200,0
4106,Female,51.0,0,0,No Info,27.32,6.6,200,0
4128,Female,80.0,0,0,never,27.32,5.7,85,0
4224,Male,80.0,1,0,former,27.32,6.2,130,0
4263,Male,80.0,0,0,No Info,27.32,6.0,100,0
4486,Male,50.0,0,0,No Info,27.32,4.8,155,0


In [389]:
dli = dliabetes[dliabetes.duplicated()]
dli[(dli['age'] == 80.0)&(dli['blood_glucose_level']==159)&(dli['HbA1c_level']==6.6)]
# 값은 같지만 원전한 중복값이라고 보기 힘든 점은 존재한다. 하지마 머신러닝에 방해가 될 수도 있다. 이에 타겟 값의 비율이나 다른 값은 평균값에 영향이 큰지 확인 할 필요가 있다.

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
2756,Male,80.0,0,0,No Info,27.32,6.6,159,0
12440,Male,80.0,0,0,No Info,27.32,6.6,159,0
47366,Female,80.0,0,0,No Info,27.32,6.6,159,0
54794,Male,80.0,0,0,No Info,27.32,6.6,159,0
75961,Male,80.0,0,0,No Info,27.32,6.6,159,0


In [390]:
dliabetes['diabetes'].value_counts()

diabetes
0    91500
1     8500
Name: count, dtype: int64

In [391]:
dliabetes['smoking_history'].value_counts()

smoking_history
No Info        35816
never          35095
former          9352
current         9286
not current     6447
ever            4004
Name: count, dtype: int64

In [392]:
list_i = ['age','bmi','HbA1c_level','blood_glucose_level']
for i in list_i:
    print(f'Columns : {i}')
    print(dliabetes[i].mean())

Columns : age
41.885856
Columns : bmi
27.3207671
Columns : HbA1c_level
5.527507
Columns : blood_glucose_level
138.05806


In [393]:
test_d = dliabetes.copy()
test_d = test_d.drop_duplicates()
test_d = test_d.reset_index(drop=True)
test_d['diabetes'].value_counts()

diabetes
0    87664
1     8482
Name: count, dtype: int64

In [394]:
for i in list_i:
    print(f'Columns : {i}')
    print(test_d[i].mean())

Columns : age
41.79432571297817
Columns : bmi
27.32146111122668
Columns : HbA1c_level
5.532608740873255
Columns : blood_glucose_level
138.21823060761758


In [395]:
list_c = ['gender', 'hypertension', 'heart_disease', 'smoking_history', 'diabetes']
for i in list_c:
    print('Colums :',dliabetes[i].value_counts())
    print('===='*10)

Colums : gender
Female    58552
Male      41430
Other        18
Name: count, dtype: int64
Colums : hypertension
0    92515
1     7485
Name: count, dtype: int64
Colums : heart_disease
0    96058
1     3942
Name: count, dtype: int64
Colums : smoking_history
No Info        35816
never          35095
former          9352
current         9286
not current     6447
ever            4004
Name: count, dtype: int64
Colums : diabetes
0    91500
1     8500
Name: count, dtype: int64


In [396]:
for i in list_c:
    print('Colums :',test_d[i].value_counts())
    print('===='*10)

Colums : gender
Female    56161
Male      39967
Other        18
Name: count, dtype: int64
Colums : hypertension
0    88685
1     7461
Name: count, dtype: int64
Colums : heart_disease
0    92223
1     3923
Name: count, dtype: int64
Colums : smoking_history
never          34398
No Info        32887
former          9299
current         9197
not current     6367
ever            3998
Name: count, dtype: int64
Colums : diabetes
0    87664
1     8482
Name: count, dtype: int64


In [397]:
df_d = dliabetes.drop_duplicates()
df_d = df_d.reset_index(drop=True)
df_d

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0
...,...,...,...,...,...,...,...,...,...
96141,Female,36.0,0,0,No Info,24.60,4.8,145,0
96142,Female,2.0,0,0,No Info,17.37,6.5,100,0
96143,Male,66.0,0,0,former,27.83,5.7,155,0
96144,Female,24.0,0,0,never,35.42,4.0,100,0


In [398]:
df_d['smoking_history'].value_counts().index

Index(['never', 'No Info', 'former', 'current', 'not current', 'ever'], dtype='object', name='smoking_history')

# 의료보험 데이터셋

In [399]:
medical.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [400]:
medical.shape

(1338, 7)

In [401]:
medical.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [402]:
medical.duplicated().sum()

1

In [403]:
df_m = medical.copy()
df_m = df_m.drop_duplicates()
df_m = df_m.reset_index(drop=True)
df_m

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1332,50,male,30.970,3,no,northwest,10600.54830
1333,18,female,31.920,0,no,northeast,2205.98080
1334,18,female,36.850,0,no,southeast,1629.83350
1335,21,female,25.800,0,no,southwest,2007.94500


In [404]:
df_m.columns

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges'], dtype='object')

In [405]:
list_c = ['sex','children', 'smoker', 'region']
for i in list_c:
    print('Colums :',df_m[i].value_counts())
    print('===='*10)

Colums : sex
male      675
female    662
Name: count, dtype: int64
Colums : children
0    573
1    324
2    240
3    157
4     25
5     18
Name: count, dtype: int64
Colums : smoker
no     1063
yes     274
Name: count, dtype: int64
Colums : region
southeast    364
southwest    325
northwest    324
northeast    324
Name: count, dtype: int64


In [406]:
list_c = ['age','bmi','charges']
for i in list_c:
    print('Colums :',i,df_m[i].mean())
    print('===='*10)

Colums : age 39.222139117427076
Colums : bmi 30.66345175766642
Colums : charges 13279.121486655948


# 데이터셋 EDA

In [407]:
df_d

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0
...,...,...,...,...,...,...,...,...,...
96141,Female,36.0,0,0,No Info,24.60,4.8,145,0
96142,Female,2.0,0,0,No Info,17.37,6.5,100,0
96143,Male,66.0,0,0,former,27.83,5.7,155,0
96144,Female,24.0,0,0,never,35.42,4.0,100,0


In [408]:
df_d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96146 entries, 0 to 96145
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   gender               96146 non-null  object 
 1   age                  96146 non-null  float64
 2   hypertension         96146 non-null  int64  
 3   heart_disease        96146 non-null  int64  
 4   smoking_history      96146 non-null  object 
 5   bmi                  96146 non-null  float64
 6   HbA1c_level          96146 non-null  float64
 7   blood_glucose_level  96146 non-null  int64  
 8   diabetes             96146 non-null  int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 6.6+ MB


In [409]:
other = list(df_d[df_d['gender'] == 'Other'].index)
other

[12589,
 14730,
 16574,
 18540,
 23023,
 31547,
 33299,
 34393,
 34466,
 39626,
 54936,
 64780,
 65054,
 65347,
 66556,
 69075,
 73002,
 73408]

In [410]:
df_d.shape

(96146, 9)

In [411]:
df_d = df_d.drop(other)
df_d.shape

(96128, 9)

In [412]:
df_d[df_d['gender'] == 'Other']

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes


In [413]:
df_d = df_d.reset_index(drop=True)
df_d

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0
...,...,...,...,...,...,...,...,...,...
96123,Female,36.0,0,0,No Info,24.60,4.8,145,0
96124,Female,2.0,0,0,No Info,17.37,6.5,100,0
96125,Male,66.0,0,0,former,27.83,5.7,155,0
96126,Female,24.0,0,0,never,35.42,4.0,100,0


In [414]:
df_d.rename(columns={'gender':'sex','smoking_history':'somker'},inplace=True)
df_d

,sex,age,hypertension,heart_disease,somker,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0
...,...,...,...,...,...,...,...,...,...
96123,Female,36.0,0,0,No Info,24.60,4.8,145,0
96124,Female,2.0,0,0,No Info,17.37,6.5,100,0
96125,Male,66.0,0,0,former,27.83,5.7,155,0
96126,Female,24.0,0,0,never,35.42,4.0,100,0


In [415]:
nan = list(df_d[df_d['somker'] == 'No Info'].index)
df_d = df_d.drop(nan)
df_d = df_d.reset_index(drop=True)

In [416]:
df_d.shape

(63247, 9)

In [417]:
# 100% 비흡연자 vs 한 번이라도 흡연을 했다.
# 비흡연자(no) : 'never', 
# 흡연자(yes) : 'former', 'current', 'not current', 'ever'
'''
- 'never' 전혀 
- 'No Info' 정보없음 -> 결측치로 보고 제거
- 'former' 이전 
- 'current' 현재 
- 'not current' 현재는 아님 
- 'ever' 항상 
'''
smoking_100_yes = ['former', 'current', 'not current', 'ever']
smoking_100_No = ['never']


In [418]:
test_d.replace(['former', 'current', 'not current', 'ever'],'Yes')

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,Yes,23.45,5.0,155,0
4,Male,76.0,1,1,Yes,20.14,4.8,155,0
...,...,...,...,...,...,...,...,...,...
96141,Female,36.0,0,0,No Info,24.60,4.8,145,0
96142,Female,2.0,0,0,No Info,17.37,6.5,100,0
96143,Male,66.0,0,0,Yes,27.83,5.7,155,0
96144,Female,24.0,0,0,never,35.42,4.0,100,0


In [419]:
df_d['sex'] = df_d['sex'].str.lower()

In [420]:
def are_you_smoking(df,yes,no):
    new_df = df.copy()
    new_df = new_df.replace(yes,'yes')
    new_df = new_df.replace(no,'no')
    return new_df

In [421]:
df_d_100 = are_you_smoking(df_d, smoking_100_yes, smoking_100_No)
df_d_100

,sex,age,hypertension,heart_disease,somker,bmi,HbA1c_level,blood_glucose_level,diabetes
0,female,80.0,0,1,no,25.19,6.6,140,0
1,male,28.0,0,0,no,27.32,5.7,158,0
2,female,36.0,0,0,yes,23.45,5.0,155,0
3,male,76.0,1,1,yes,20.14,4.8,155,0
4,female,20.0,0,0,no,27.32,6.6,85,0
...,...,...,...,...,...,...,...,...,...
63242,female,26.0,0,0,no,34.34,6.5,160,0
63243,female,40.0,0,0,no,40.69,3.5,155,0
63244,male,66.0,0,0,yes,27.83,5.7,155,0
63245,female,24.0,0,0,no,35.42,4.0,100,0


In [422]:
# 현재를 기준으로 비흡연자 vs 흡연자
# 비흡연자(no) : 'never', 'former', 'not current'
# 흡연자(yes) : 'ever','current'
smoking_yes = ['ever','current']
smoking_no = ['never', 'former', 'not current']

In [423]:
df_d_n = are_you_smoking(df_d,smoking_yes,smoking_no)
df_d_n

,sex,age,hypertension,heart_disease,somker,bmi,HbA1c_level,blood_glucose_level,diabetes
0,female,80.0,0,1,no,25.19,6.6,140,0
1,male,28.0,0,0,no,27.32,5.7,158,0
2,female,36.0,0,0,yes,23.45,5.0,155,0
3,male,76.0,1,1,yes,20.14,4.8,155,0
4,female,20.0,0,0,no,27.32,6.6,85,0
...,...,...,...,...,...,...,...,...,...
63242,female,26.0,0,0,no,34.34,6.5,160,0
63243,female,40.0,0,0,no,40.69,3.5,155,0
63244,male,66.0,0,0,no,27.83,5.7,155,0
63245,female,24.0,0,0,no,35.42,4.0,100,0


In [424]:
list_c = ['sex','somker']
for i in list_c:
    print('Colums :',df_d_100[i].value_counts())
    print('===='*10)

Colums : sex
female    38192
male      25055
Name: count, dtype: int64
Colums : somker
no     34395
yes    28852
Name: count, dtype: int64


In [425]:
for i in list_c:
    print('Colums :',df_d_n[i].value_counts())
    print('===='*10)

Colums : sex
female    38192
male      25055
Name: count, dtype: int64
Colums : somker
no     50053
yes    13194
Name: count, dtype: int64


In [426]:
df_m

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1332,50,male,30.970,3,no,northwest,10600.54830
1333,18,female,31.920,0,no,northeast,2205.98080
1334,18,female,36.850,0,no,southeast,1629.83350
1335,21,female,25.800,0,no,southwest,2007.94500


# 머신러닝 모델 제작

In [427]:
from sklearn.model_selection import train_test_split
from category_encoders import OrdinalEncoder
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import random


In [428]:
random.seed(42)
np,random.seed(42)

(<module 'numpy' from '/Users/bumsoojoe/anaconda3/envs/project/lib/python3.8/site-packages/numpy/__init__.py'>,
 None)

In [429]:
df_d_100

,sex,age,hypertension,heart_disease,somker,bmi,HbA1c_level,blood_glucose_level,diabetes
0,female,80.0,0,1,no,25.19,6.6,140,0
1,male,28.0,0,0,no,27.32,5.7,158,0
2,female,36.0,0,0,yes,23.45,5.0,155,0
3,male,76.0,1,1,yes,20.14,4.8,155,0
4,female,20.0,0,0,no,27.32,6.6,85,0
...,...,...,...,...,...,...,...,...,...
63242,female,26.0,0,0,no,34.34,6.5,160,0
63243,female,40.0,0,0,no,40.69,3.5,155,0
63244,male,66.0,0,0,yes,27.83,5.7,155,0
63245,female,24.0,0,0,no,35.42,4.0,100,0


In [430]:
train_d100, test_d100 = train_test_split(df_d_100, test_size=0.2, stratify= df_d_100['diabetes'], random_state=42)
train_1d100, val_d100 = train_test_split(train_d100, test_size=0.2, stratify= train_d100['diabetes'], random_state=42)

train_d100.shape, val_d100.shape, test_d100.shape

((50597, 9), (10120, 9), (12650, 9))

In [431]:
train_dy_100 = train_d100['diabetes']
train_dx_100 = train_d100.drop(columns=['diabetes','HbA1c_level','blood_glucose_level'],axis=1)

val_dy_100 = val_d100['diabetes']
val_dx_100 = val_d100.drop(columns=['diabetes','HbA1c_level','blood_glucose_level'],axis=1)

test_dy_100 = test_d100['diabetes']
test_dx_100 = test_d100.drop(columns=['diabetes','HbA1c_level','blood_glucose_level'],axis=1)

print('train : ', train_dy_100.shape, train_dx_100.shape)
print('val : ', val_dy_100.shape, val_dx_100.shape)
print('test : ', test_dy_100.shape, test_dx_100.shape)

train :  (50597,) (50597, 6)
val :  (10120,) (10120, 6)
test :  (12650,) (12650, 6)


In [432]:
pipe = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(strategy="median"),
    XGBClassifier(
        objective="binary:logistic",
        eval_metric="error",
        n_estimators=200,
        random_state=42,
        n_jobs=-1,
        max_depth=7,
        learning_rate=0.1,
        use_label_encoder=False,
    ),
)
print(pipe)

Pipeline(steps=[('ordinalencoder', OrdinalEncoder()),
                ('simpleimputer', SimpleImputer(strategy='median')),
                ('xgbclassifier',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric='error',
                               feature_types=None, gamma=None, gpu_id=None,
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None, learning_rate=0.1,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=7, max_leaves=None,
                               min_child_weight=None, missing=n

/Users/bumsoojoe/anaconda3/envs/project/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


In [433]:
params = {
    "xgbclassifier__max_depth": [2, 4, 6],
    "xgbclassifier__min_child_weight": [2, 4, 8],
    "xgbclassifier__colsample_bytree": [0.6, 0.8, 1.0],
}

In [434]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(pipe, param_grid=params, scoring="roc_auc", cv=3, verbose=3)
grid_search.fit(train_dx_100, train_dy_100)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


/Users/bumsoojoe/anaconda3/envs/project/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/Users/bumsoojoe/anaconda3/envs/project/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[CV 1/3] END xgbclassifier__colsample_bytree=0.6, xgbclassifier__max_depth=2, xgbclassifier__min_child_weight=2;, score=0.809 total time=   0.4s


/Users/bumsoojoe/anaconda3/envs/project/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[CV 2/3] END xgbclassifier__colsample_bytree=0.6, xgbclassifier__max_depth=2, xgbclassifier__min_child_weight=2;, score=0.814 total time=   0.3s


...

[CV 2/3] END xgbclassifier__colsample_bytree=1.0, xgbclassifier__max_depth=6, xgbclassifier__min_child_weight=8;, score=0.808 total time=   0.9s


/Users/bumsoojoe/anaconda3/envs/project/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[CV 3/3] END xgbclassifier__colsample_bytree=1.0, xgbclassifier__max_depth=6, xgbclassifier__min_child_weight=8;, score=0.804 total time=   0.9s


/Users/bumsoojoe/anaconda3/envs/project/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('ordinalencoder', OrdinalEncoder()),
                                       ('simpleimputer',
                                        SimpleImputer(strategy='median')),
                                       ('xgbclassifier',
                                        XGBClassifier(base_score=None,
                                                      booster=None,
                                                      callbacks=None,
                                                      colsample_bylevel=None,
                                                      colsample_bynode=None,
                                                      colsample_bytree=None,
                                                      early_stopping_rounds=None,
                                                      enable_categorical=False,
                                                      eval_metric='error',
                                                      featu...
                                                      max_delta_step=None,
                                                      max_depth=7,
                                                      max_leaves=None,
                                                      min_child_weight=None,
                                                      missing=nan,
                                                      monotone_constraints=None,
                                                      n_estimators=200,
                                                      n_jobs=-1,
                                                      num_parallel_tree=None,
                                                      predictor=None,
                                                      random_state=42, ...))]),
             param_grid={'xgbclassifier__colsample_bytree': [0.6, 0.8, 1.0],
                         'xgbclassifier__max_depth': [2, 4, 6],
                         'xgbclassifier__min_child_weight': [2, 4, 8]},
             scoring='roc_auc', verbose=3)

In [435]:
print("최적 하이퍼파라미터: ", grid_search.best_params_)
print("최적 AUC: ", grid_search.best_score_)

최적 하이퍼파라미터:  {'xgbclassifier__colsample_bytree': 0.6, 'xgbclassifier__max_depth': 2, 'xgbclassifier__min_child_weight': 2}
최적 AUC:  0.8099000491336464


In [436]:
from sklearn.metrics import classification_report

print("검증 정확도", grid_search.score(val_dx_100, val_dy_100))

y_pred = grid_search.predict(val_dx_100)
print(classification_report(val_dy_100, y_pred))

검증 정확도 0.8195130382955125
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      8994
           1       0.50      0.06      0.11      1126

    accuracy                           0.89     10120
   macro avg       0.70      0.53      0.53     10120
weighted avg       0.85      0.89      0.85     10120



In [437]:
from scipy.stats.distributions import uniform

params = {
    "simpleimputer__strategy": ["median", "mean"],
    "xgbclassifier__max_depth": [2, 4, 6],
    "xgbclassifier__min_child_weight": [2, 4, 8],
    "xgbclassifier__colsample_bytree": uniform(
        loc=0.5, scale=0.5
    ),  # 0.5 ~ 1 사이의 uniform 분포로 범위를 지정해 줍니다.
}

In [438]:
from sklearn.model_selection import RandomizedSearchCV

randomized_search = RandomizedSearchCV(
    pipe,
    param_distributions=params,
    scoring="roc_auc",
    n_iter=10,
    cv=3,
    verbose=3,
    random_state=42,
)
randomized_search.fit(train_dx_100, train_dy_100)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


/Users/bumsoojoe/anaconda3/envs/project/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/Users/bumsoojoe/anaconda3/envs/project/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[CV 1/3] END simpleimputer__strategy=median, xgbclassifier__colsample_bytree=0.8982714934301164, xgbclassifier__max_depth=6, xgbclassifier__min_child_weight=8;, score=0.805 total time=   0.8s


/Users/bumsoojoe/anaconda3/envs/project/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[CV 2/3] END simpleimputer__strategy=median, xgbclassifier__colsample_bytree=0.8982714934301164, xgbclassifier__max_depth=6, xgbclassifier__min_child_weight=8;, score=0.808 total time=   0.8s


...

[CV 2/3] END simpleimputer__strategy=mean, xgbclassifier__colsample_bytree=0.8925879806965068, xgbclassifier__max_depth=6, xgbclassifier__min_child_weight=8;, score=0.808 total time=   0.8s


/Users/bumsoojoe/anaconda3/envs/project/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[CV 3/3] END simpleimputer__strategy=mean, xgbclassifier__colsample_bytree=0.8925879806965068, xgbclassifier__max_depth=6, xgbclassifier__min_child_weight=8;, score=0.802 total time=   0.8s


/Users/bumsoojoe/anaconda3/envs/project/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('ordinalencoder',
                                              OrdinalEncoder()),
                                             ('simpleimputer',
                                              SimpleImputer(strategy='median')),
                                             ('xgbclassifier',
                                              XGBClassifier(base_score=None,
                                                            booster=None,
                                                            callbacks=None,
                                                            colsample_bylevel=None,
                                                            colsample_bynode=None,
                                                            colsample_bytree=None,
                                                            early_stopping_rounds=None,
                                                            enable_categorical=False,
                                                            eval_metric='error'...
                                                            num_parallel_tree=None,
                                                            predictor=None,
                                                            random_state=42, ...))]),
                   param_distributions={'simpleimputer__strategy': ['median',
                                                                    'mean'],
                                        'xgbclassifier__colsample_bytree': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x2bfb48b80>,
                                        'xgbclassifier__max_depth': [2, 4, 6],
                                        'xgbclassifier__min_child_weight': [2,
                                                                            4,
                                                                            8]},
                   random_state=42, scoring='roc_auc', verbose=3)

In [439]:
print("최적 하이퍼파라미터: ", randomized_search.best_params_)
print("최적 AUC: ", randomized_search.best_score_)

최적 하이퍼파라미터:  {'simpleimputer__strategy': 'mean', 'xgbclassifier__colsample_bytree': 0.5909124836035503, 'xgbclassifier__max_depth': 2, 'xgbclassifier__min_child_weight': 2}
최적 AUC:  0.8099000491336464


In [440]:
print("검증 정확도", randomized_search.score(val_dx_100, val_dy_100))

y_pred = randomized_search.predict(val_dx_100)
print(classification_report(val_dy_100, y_pred))

검증 정확도 0.8195130382955125
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      8994
           1       0.50      0.06      0.11      1126

    accuracy                           0.89     10120
   macro avg       0.70      0.53      0.53     10120
weighted avg       0.85      0.89      0.85     10120



In [441]:
train_d, test_d = train_test_split(df_d_n, test_size=0.2, stratify= df_d_n['diabetes'], random_state=42)
train_d, val_d = train_test_split(train_d, test_size=0.2, stratify= train_d['diabetes'], random_state=42)

train_dy = train_d['diabetes']
train_dx = train_d.drop(columns=['diabetes','HbA1c_level','blood_glucose_level','hypertension','heart_disease'],axis=1)

val_dy = val_d['diabetes']
val_dx = val_d.drop(columns=['diabetes','HbA1c_level','blood_glucose_level','hypertension','heart_disease'],axis=1)

test_dy = test_d['diabetes']
test_dx= test_d.drop(columns=['diabetes','HbA1c_level','blood_glucose_level','hypertension','heart_disease'],axis=1)

print('train : ', train_dy.shape, train_dx.shape)
print('val : ', val_dy.shape, val_dx.shape)
print('test : ', test_dy.shape, test_dx.shape)

train :  (40477,) (40477, 4)
val :  (10120,) (10120, 4)
test :  (12650,) (12650, 4)


In [442]:
pipe_xbg = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(strategy="median"),
    XGBClassifier(
        objective="binary:logistic",
        eval_metric="error",
        n_estimators=200,
        random_state=42,
        n_jobs=-1,
        max_depth=7,
        learning_rate=0.1,
        use_label_encoder=False,
    ),
)
print(pipe_xbg)

Pipeline(steps=[('ordinalencoder', OrdinalEncoder()),
                ('simpleimputer', SimpleImputer(strategy='median')),
                ('xgbclassifier',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric='error',
                               feature_types=None, gamma=None, gpu_id=None,
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None, learning_rate=0.1,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=7, max_leaves=None,
                               min_child_weight=None, missing=n

/Users/bumsoojoe/anaconda3/envs/project/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


In [443]:
pipe_xbg.fit(train_dx, train_dy)

Pipeline(steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['sex', 'somker'],
                                mapping=[{'col': 'sex', 'data_type': dtype('O'),
                                          'mapping': male      1
female    2
NaN      -2
dtype: int64},
                                         {'col': 'somker',
                                          'data_type': dtype('O'),
                                          'mapping': no     1
yes    2
NaN   -2
dtype: int64}])),
                ('simpleimputer', SimpleImputer(strategy='median')),
                ('xgbclassifier',
                 XGBClassifier(base_score=0.5, booster='gbtree', cal...
                               feature_types=None, gamma=0, gpu_id=-1,
                               grow_policy='depthwise', importance_type=None,
                               interaction_constraints='', learning_rate=0.1,
                               max_bin=256, max_cat_threshold=64,
                               max_cat_to_onehot=4, max_delta_step=0,
                               max_depth=7, max_leaves=0, min_child_weight=1,
                               missing=nan, monotone_constraints='()',
                               n_estimators=200, n_jobs=-1, num_parallel_tree=1,
                               predictor='auto', random_state=42, ...))])

In [444]:
print("검증 정확도", pipe_xbg.score(val_dx, val_dy))

y_pred = pipe_xbg.predict(val_dx)
print(classification_report(val_dy, y_pred))
# 흡연자는 현재를 기준이 당뇨환자를 찾아내는데 중요한다.

검증 정확도 0.8872529644268775
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      8994
           1       0.44      0.05      0.09      1126

    accuracy                           0.89     10120
   macro avg       0.67      0.52      0.52     10120
weighted avg       0.84      0.89      0.85     10120



In [445]:
from sklearn.ensemble import RandomForestClassifier

pipe_rf = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(strategy="median"),
    RandomForestClassifier(
        random_state=42,
        oob_score=True,
        n_jobs=-1,),)
print(pipe_rf)

Pipeline(steps=[('ordinalencoder', OrdinalEncoder()),
                ('simpleimputer', SimpleImputer(strategy='median')),
                ('randomforestclassifier',
                 RandomForestClassifier(n_jobs=-1, oob_score=True,
                                        random_state=42))])


In [446]:
pipe_rf.fit(train_dx, train_dy)

Pipeline(steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['sex', 'somker'],
                                mapping=[{'col': 'sex', 'data_type': dtype('O'),
                                          'mapping': male      1
female    2
NaN      -2
dtype: int64},
                                         {'col': 'somker',
                                          'data_type': dtype('O'),
                                          'mapping': no     1
yes    2
NaN   -2
dtype: int64}])),
                ('simpleimputer', SimpleImputer(strategy='median')),
                ('randomforestclassifier',
                 RandomForestClassifier(n_jobs=-1, oob_score=True,
                                        random_state=42))])

In [447]:
print("검증 정확도", pipe_rf.score(val_dx, val_dy))

y_pred = pipe_rf.predict(val_dx)
print(classification_report(val_dy, y_pred))

검증 정확도 0.8590909090909091
              precision    recall  f1-score   support

           0       0.90      0.94      0.92      8994
           1       0.29      0.18      0.22      1126

    accuracy                           0.86     10120
   macro avg       0.60      0.56      0.57     10120
weighted avg       0.83      0.86      0.84     10120



In [448]:
df_m

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1332,50,male,30.970,3,no,northwest,10600.54830
1333,18,female,31.920,0,no,northeast,2205.98080
1334,18,female,36.850,0,no,southeast,1629.83350
1335,21,female,25.800,0,no,southwest,2007.94500


In [449]:
train_dx

,sex,age,somker,bmi
35720,male,20.0,no,24.66
43583,male,80.0,no,30.79
43416,female,40.0,no,22.92
44395,male,8.0,no,17.56
42347,male,80.0,no,27.21
...,...,...,...,...
13992,male,69.0,no,24.71
62687,female,43.0,yes,41.29
37115,male,40.0,no,25.05
33570,male,65.0,yes,26.36


In [450]:
df_m = df_m[['sex','age','smoker','bmi','children','region','charges']]
df_m

,sex,age,smoker,bmi,children,region,charges
0,female,19,yes,27.900,0,southwest,16884.92400
1,male,18,no,33.770,1,southeast,1725.55230
2,male,28,no,33.000,3,southeast,4449.46200
3,male,33,no,22.705,0,northwest,21984.47061
4,male,32,no,28.880,0,northwest,3866.85520
...,...,...,...,...,...,...,...
1332,male,50,no,30.970,3,northwest,10600.54830
1333,female,18,no,31.920,0,northeast,2205.98080
1334,female,18,no,36.850,0,southeast,1629.83350
1335,female,21,no,25.800,0,southwest,2007.94500


In [455]:
df_m.columns

Index(['sex', 'age', 'smoker', 'bmi', 'children', 'region', 'charges'], dtype='object')

In [454]:
df_m.columns

Index(['sex', 'age', 'smoker', 'bmi', 'children', 'region', 'charges'], dtype='object')

In [456]:
df_m_d = df_m[['sex','age','smoker','bmi']]
df_m_d = df_m_d.replace('yes','yes')
df_m_d = df_m_d.replace('no','no')
df_m_d.rename(columns={'smoker':'smoker'},inplace=True)
df_m_d

,sex,age,smoker,bmi
0,female,19,yes,27.900
1,male,18,no,33.770
2,male,28,no,33.000
3,male,33,no,22.705
4,male,32,no,28.880
...,...,...,...,...
1332,male,50,no,30.970
1333,female,18,no,31.920
1334,female,18,no,36.850
1335,female,21,no,25.800


In [458]:
medical_pred = pipe_xbg.predict(df_m_d)
medical_pred

KeyError: 'somker'